In [2]:
%pip install beautifulsoup4
%pip install lxml
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import urllib.request as rq
from bs4 import BeautifulSoup
import pandas as pd
import re

<h1>QUESTION 1</h1>

<h1>scavenge data from Wikipedia</h1>

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = rq.urlopen(url)
soup = BeautifulSoup(page, "lxml")

<h1>retrieve information from raw Wikipedia's data</h1>

In [5]:
#retrieve table
all_tables = soup.find_all('table')
right_table=soup.find('table', class_='wikitable sortable')
#take data
Postcode_data = []
Borough_data = []
Neighborhood_data = []
for row in right_table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells) == 3:
        Postcode_data.append(cells[0].find(text=True))
        Borough_data.append(cells[1].find(text=True))
        Neighborhood_data.append(cells[2].find(text=True))

df = pd.DataFrame({'PostalCode':Postcode_data, 'Borough':Borough_data, 'Neighborhood':Neighborhood_data})
df

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West\n
285,M8Z,Etobicoke,South of Bloor\n


<h1>Preprocessing</h1>

<p>drop all 'Not assigned' value, fix value containing '\n' and add value to Neighborhood if not yet exist </p>

In [6]:
#preprocessing
#drop all 'Not assigned' value and fix value containing '\n'
counter=0
while counter < df.shape[0]:
    for j in range(1,3):
        if re.search('Not assigned.*',df.iloc[counter,j]) != None:
            if j == 1:
                df.drop(df.index[counter], axis=0, inplace=True)
                counter -= 1
            if j == 2:
                df.iloc[counter,2] = df.iloc[counter,1]
            break
            
        if re.search('.*\n',df.iloc[counter,j]) != None:
            df.iloc[counter,j] = df.iloc[counter,j][0:len(df.iloc[counter,j])-2]
        
    counter += 1

<p>merge duplicates</p>

In [7]:
#merge duplicates
size = df.shape[0]
unique = df['PostalCode'].unique()
borough = []
neighbors = []

count = -1
current = ''
for b in range(size):
    if df.iloc[b,0] == current:
        neighbors[count] += ', ' + df.iloc[b,2]
    else:
        current = df.iloc[b,0]
        neighbors.append(df.iloc[b,2])
        borough.append(df.iloc[b,1])
        count += 1

df = pd.DataFrame({'PostalCode':unique, 'Borough':borough, 'Neighborhood':neighbors})
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Roa, Old Mill Nort"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Easter
101,M8Y,Etobicoke,"Humber Bay, King's Mill Par, Kingsway Park Sou..."


<p>check if there are duplicates</p>

In [8]:
#check if there are duplicates
bool_series = df.duplicated('PostalCode').values
duplicates = 0
for i in bool_series:
    if i == True:
        duplicates += 1
        
print('Duplicates = {0}'.format(duplicates))

Duplicates = 0


<h1>Show Shape</h1>

In [9]:
print('Data Frame shape = {0}'.format(df.shape))

Data Frame shape = (103, 3)


<h1>QUESTION 2</h1>

<p>Get coordinate</p>

In [10]:
url = 'http://cocl.us/Geospatial_data'
df_location = pd.read_csv(url)
df_location

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


<p>Form new Data Frame containing coordinate</p>

In [11]:
df.sort_values('PostalCode', inplace=True)
merged_df = df.merge(df_location, left_on='PostalCode', right_on='Postal Code')
merged_df

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwoo, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbra,M1H,43.773136,-79.239476
...,...,...,...,...,...,...
98,M9N,York,Weston,M9N,43.706876,-79.518188
99,M9P,Etobicoke,Westmoun,M9P,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Garden, Richvi...",M9R,43.688905,-79.554724
101,M9V,Etobicoke,"Albion Garden, Beaumond Heights, Humbergat, Ja...",M9V,43.739416,-79.588437


<h1>Question 3</h1>

<p>Download necessary package</p>

In [12]:
%pip install folium

Note: you may need to restart the kernel to use updated packages.


In [17]:
import folium
from folium.plugins import MarkerCluster

<p>Initiate map of Toronto and add markers to map</p>

In [14]:
toronto_latitude = 43.6529
toronto_longitude = -79.3849
map_toronto = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start = 10)

# add markers to map
for lat, lng, borough, neighborhood in zip(merged_df['Latitude'], merged_df['Longitude'], merged_df['Borough'], merged_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 

In [15]:
map_toronto

Cluster Neighborhood

In [18]:
map_toronto = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start = 10)

cluster_groups = MarkerCluster().add_to(map_toronto)

for lat, lng, borough, neighborhood in zip(merged_df['Latitude'], merged_df['Longitude'], merged_df['Borough'], merged_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(cluster_groups) 
    
map_toronto